In [ ]:
import os

# Data paths
dir_local = '/home/study/IdeaProjects/Graph-Machine-Learning/Temporal_RSR/data'
dir_kaggle = '/kaggle/input/rsr-dataset/Data/'

num_companies = 1026  # max is 1026
num_days = 1245
num_features = 5
market = "NASDAQ"  # or "NYSE"

train_size = 756
validation_size = 252
test_size = 237

# Temporal parameters
window_size = 20  # Lookback period for temporal models
prediction_horizon = 1  # How many days ahead to predict

BASE_MODEL_CONFIG = {
    'use_gat': True,        # GAT model (attention-based)
    
    # Graph features
    'use_industry_relations': True,   # Use industry relationship data
    'use_wiki_relations': True,       # Use Wikipedia relationship data
    'use_graph_structure': True,      # If False, uses identity matrix (no graph)
    'separate_edge_weights': True,    # If True, use 2D edge features [industry_wt, wiki_wt]
    
    # Model architecture
    'hidden_dim': 16,                 # Hidden layer dimension for GAT embeddings
    'num_attention_heads': 3,         # For GAT model
    'dropout': 0.0,                   # Dropout rate
    'use_relu_output': False,         # Apply ReLU after final layer (False for regression)
    'use_graph_concatenation': True,   # Use graph concatenation, this allows for multiple graphs all processed together at once
    
    # Temporal modeling (RNN)
    'use_rnn': False,                  # Add RNN layer after GAT for temporal modeling
    'use_rnn_first': False,            # Use RNN before GAT layer, otherwise after
    'rnn_type': 'LSTM',               # 'LSTM' or 'GRU'
    'rnn_hidden_dim': 16,             # RNN hidden state dimension
    'rnn_num_layers': 1,              # Number of RNN layers
    
    # Training parameters
    'learning_rate': 0.001,
    'weight_decay': 0.0,              # L2 regularization
    'batch_size': 4,                  # batch size for training
}

CONFIGS_TO_TEST = [
    # Baseline: No temporal modeling
    {'name': 'GAT_only', 'use_rnn': False},
    
    # RNN variants - GAT first then RNN
    # {'name': 'GAT_LSTM', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM'},
    # {'name': 'GAT_GRU', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'GRU'},

    {'name': 'GAT_LSTM', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM'},

    {'name': 'GAT_LSTM_attention_heads_1', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM', 'hidden_dim': 16, "num_attention_heads": 1},
    {'name': 'GAT_LSTM_attention_heads_2', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM', 'hidden_dim': 16, "num_attention_heads": 2},
    {'name': 'GAT_LSTM_attention_heads_3', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM', 'hidden_dim': 16, "num_attention_heads": 3},
    {'name': 'GAT_LSTM_attention_heasd_4', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM', 'hidden_dim': 16, "num_attention_heads": 4},
    # Different hidden dimensions
    {'name': 'GAT_LSTM_h16', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM', 'hidden_dim': 16},
    {'name': 'GAT_LSTM_h32', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM', 'hidden_dim': 32,},
    # RNN variants - RNN first then GAT  
    # {'name': 'LSTM_GAT', 'use_rnn': True, 'use_rnn_first': True, 'rnn_type': 'LSTM'},
    # {'name': 'GRU_GAT', 'use_rnn': True, 'use_rnn_first': True, 'rnn_type': 'GRU'},

    # Different hidden dimensions for LSTM
    {'name': 'GAT_LSTM_h16', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM', 'hidden_dim': 16, 'rnn_hidden_dim': 16},
    {'name': 'GAT_LSTM_h16', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM', 'hidden_dim': 16, 'rnn_hidden_dim': 32},
    {'name': 'GAT_LSTM_h16', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM', 'hidden_dim': 16, 'rnn_hidden_dim': 64},
    {'name': 'GAT_LSTM_h16', 'use_rnn': True, 'use_rnn_first': False, 'rnn_type': 'LSTM', 'hidden_dim': 16, 'rnn_hidden_dim': 128},
    
    # Different learning rates
    {'name': 'GAT_LSTM_lr0.01', 'use_rnn': True, 'use_rnn_first': False, 'learning_rate': 0.01},
    {'name': 'GAT_LSTM_lr0.0001', 'use_rnn': True, 'use_rnn_first': False, 'learning_rate': 0.0001},
]

# Working configuration (will be updated during grid search)
MODEL_CONFIG = BASE_MODEL_CONFIG.copy()

epochs = 1000
early_stopping_patience = 100  # Stop if no improvement for N epochs
the_max_batch_size = 32  # Max batch size that is used in configs

# Environment detection
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    print("Running on Kaggle!")
    dir = dir_kaggle
    epochs = 1000
    num_companies = 1026
else:
    dir = dir_local
    print("Running locally!")

print(f"\n{'='*60}")
print(f"GRID SEARCH CONFIGURATION")
print(f"{'='*60}")
print(f"Dataset:")
print(f"  Market: {market}")
print(f"  Companies: {num_companies}")
print(f"  Features: {num_features}")
print(f"  Window size: {window_size}")
print(f"  Prediction horizon: {prediction_horizon}")

print(f"\nData Split: default based on the one from Temporal Relational Stock Ranking paper")
print(f"\nGrid Search:")
print(f"  Number of configurations: {len(CONFIGS_TO_TEST)}")
print(f"  Configurations to test:")
for i, cfg in enumerate(CONFIGS_TO_TEST, 1):
    print(f"    {i}. {cfg['name']}")

print(f"\nTraining:")
print(f"  Epochs per config: {epochs}")
print(f"  Early stopping patience: {early_stopping_patience}")
print(f"{'='*60}\n")

In [ ]:
%pip install torch-geometric
%pip install statsmodels


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torch_geometric.nn import GATConv
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

In [ ]:
"""
COPIED FROM THE PAPER
source code: https://github.com/fulifeng/Temporal_Relational_Stock_Ranking
"""
def load_EOD_data(data_path, market_name, tickers, steps=1):
    eod_data = []
    masks = []
    ground_truth = []
    base_price = []

    # Determine the expected number of rows based on the first ticker's data
    first_ticker_path = os.path.join(data_path, market_name + '_' + tickers[0] + '_1.csv')
    try:
        first_df = pd.read_csv(first_ticker_path, header=None)
        num_days = first_df.shape[0] - (1 if market_name == 'NASDAQ' else 0) # Remove last row for NASDAQ
        num_features = first_df.shape[1] - 1 # Exclude the date column
    except Exception as e:
        print(f"Error reading first ticker file {first_ticker_path}: {e}")
        return None, None, None, None

    eod_data = np.zeros([len(tickers), num_days, num_features], dtype=np.float32)
    masks = np.ones([len(tickers), num_days], dtype=np.float32)
    ground_truth = np.zeros([len(tickers), num_days], dtype=np.float32) # We're not using this one
    base_price = np.zeros([len(tickers), num_days], dtype=np.float32)

    for index, ticker in enumerate(tickers):
        if index % 50 == 0:
          print(f"Processed [{index}/{tickers.shape[0]}] tickers")
        single_EOD_path = os.path.join(data_path, market_name + '_' + ticker + '_1.csv')

        try:
            single_df = pd.read_csv(single_EOD_path, header=None)
            if market_name == 'NASDAQ':
                single_df = single_df[:-1] # remove the last day since lots of missing data

            # Handle missing values (-1234)
            single_EOD = single_df.values
            mask_row_indices, mask_col_indices = np.where(np.abs(single_EOD + 1234) < 1e-8)
            single_EOD[mask_row_indices, mask_col_indices] = 1.1 # Replace missing values

            # Update masks based on missing closing price
            missing_close_indices = np.where(np.abs(single_EOD[:, -1] + 1234) < 1e-8)[0]
            masks[index, missing_close_indices] = 0.0

            eod_data[index, :, :] = single_EOD[:, 1:] # Exclude date column
            base_price[index, :] = single_EOD[:, -1]

        except Exception as e:
            print(f"Error reading ticker file {single_EOD_path}: {e}")
            # Mark all days for this ticker as invalid if file reading fails
            masks[index, :] = 0.0


    print('eod data shape:', eod_data.shape)
    return eod_data, masks, ground_truth, base_price

In [ ]:
"""
COPIED FROM THE PAPER
source code: https://github.com/fulifeng/Temporal_Relational_Stock_Ranking
"""
def load_relation_data(relation_file):
    relation_encoding = np.load(relation_file)
    print('relation encoding shape:', relation_encoding.shape)
    rel_shape = [relation_encoding.shape[0], relation_encoding.shape[1]]
    mask_flags = np.equal(np.zeros(rel_shape, dtype=int),
                          np.sum(relation_encoding, axis=2))
    mask = np.where(mask_flags, np.ones(rel_shape) * -1e9, np.zeros(rel_shape))
    return relation_encoding, mask

# Loading data

In [ ]:
industry_encodings, industry_mask = load_relation_data(dir+f'/relation/sector_industry/{market}_industry_relation.npy')

In [ ]:
wiki_encodings, wiki_mask = load_relation_data(dir+f'/relation/wikidata/{market}_wiki_relation.npy')

In [ ]:
# Load company names
tickers = np.loadtxt(dir+f'/{market}_tickers.csv', dtype=str)
print('tickers shape (# of companies):', tickers.shape)

In [ ]:
print(f"Subsampling to {num_companies} companies...")

# Subsample relation data (company x company matrices)
wiki_encodings = wiki_encodings[:num_companies, :num_companies, :]
wiki_mask = wiki_mask[:num_companies, :num_companies]
industry_encodings = industry_encodings[:num_companies, :num_companies, :]
industry_mask = industry_mask[:num_companies, :num_companies]

# Subsample EOD data (reload with subset of tickers)
eod_data, eod_masks, eod_ground_truth, eod_base_price = load_EOD_data(
    dir + "/2013-01-01", 
    market, 
    tickers[:num_companies]
)

print(f"\nSubsampled data shapes:")
print(f"  EOD data: {eod_data.shape}")
print(f"  Industry encodings: {industry_encodings.shape}")
print(f"  Wiki encodings: {wiki_encodings.shape}")

# Utils

In [ ]:
def build_adjacency_matrix(industry_encodings, industry_mask, wiki_encodings, wiki_mask, device):
    """
    Build normalized adjacency matrix from relation encodings and masks
    
    Args:
        industry_encodings: [num_companies, num_companies, num_relation_types]
        industry_mask: [num_companies, num_companies] (-1e9 for no relation, 0 for valid)
        wiki_encodings: [num_companies, num_companies, num_relation_types]
        wiki_mask: [num_companies, num_companies]
    
    Returns:
        adjacency_matrix: [num_companies, num_companies] - normalized adjacency
    """
    # Combine relation encodings by summing across relation types
    industry_adj = torch.sum(industry_encodings, dim=-1) if MODEL_CONFIG['use_industry_relations'] else 0
    wiki_adj = torch.sum(wiki_encodings, dim=-1) if MODEL_CONFIG['use_wiki_relations'] else 0
    
    # Combine both relation types
    combined_adj = industry_adj + wiki_adj
    
    # Apply masks: where mask is -1e9 (no relation), set adjacency to 0
    combined_mask = industry_mask + wiki_mask
    combined_adj = torch.where(combined_mask < -1e8, torch.zeros_like(combined_adj), combined_adj)
    
    # If not using graph structure, return identity matrix
    if not MODEL_CONFIG['use_graph_structure']:
        return torch.eye(combined_adj.shape[0], device=device)
    
    # Normalize: row-wise normalization (each row sums to 1)
    row_sums = combined_adj.sum(dim=1, keepdim=True)
    adjacency_matrix = combined_adj / (row_sums + 1e-8)
    
    return adjacency_matrix.to(device)


def build_separate_adjacency_matrices(industry_encodings, industry_mask, wiki_encodings, wiki_mask, device):
    """
    Build SEPARATE normalized adjacency matrices for industry and wiki relations
    Used when separate_edge_weights=True
    
    Returns:
        industry_adj: [num_companies, num_companies]
        wiki_adj: [num_companies, num_companies]
    """
    # Build industry adjacency
    industry_adj = torch.sum(industry_encodings, dim=-1) if MODEL_CONFIG['use_industry_relations'] else torch.zeros_like(industry_mask)
    industry_adj = torch.where(industry_mask < -1e8, torch.zeros_like(industry_adj), industry_adj)
    
    # Build wiki adjacency
    wiki_adj = torch.sum(wiki_encodings, dim=-1) if MODEL_CONFIG['use_wiki_relations'] else torch.zeros_like(wiki_mask)
    wiki_adj = torch.where(wiki_mask < -1e8, torch.zeros_like(wiki_adj), wiki_adj)
    
    # If not using graph structure, return identity matrices
    if not MODEL_CONFIG['use_graph_structure']:
        identity = torch.eye(industry_adj.shape[0], device=device)
        return identity, identity
    
    # Normalize each separately
    industry_row_sums = industry_adj.sum(dim=1, keepdim=True)
    industry_adj = industry_adj / (industry_row_sums + 1e-8)
    
    wiki_row_sums = wiki_adj.sum(dim=1, keepdim=True)
    wiki_adj = wiki_adj / (wiki_row_sums + 1e-8)
    
    return industry_adj.to(device), wiki_adj.to(device)

def prepare_data(eod_data, masks, base_price, device, window_size=20, prediction_horizon=1):
    """
    Create sliding windows for time series prediction with mask handling
    
    Args:
        eod_data: [num_companies, num_days, num_features]
        masks: [num_companies, num_days] - 1.0 for valid, 0.0 for missing
        base_price: [num_companies, num_days] - closing price of stock
        window_size: Number of historical days to use as input
        prediction_horizon: Number of days ahead to predict (usually 1)
    
    Returns:
        X: Input windows [num_samples, num_companies, window_size, num_features]
        y: Target prices [num_samples, num_companies, prediction_horizon]
        sample_masks: Valid sample indicators [num_samples, num_companies]
    """
    num_companies, num_days, num_features = eod_data.shape
    num_samples = num_days - window_size - prediction_horizon + 1
    
    X = torch.zeros(num_samples, num_companies, window_size, num_features, device=device)
    y = torch.zeros(num_samples, num_companies, prediction_horizon, device=device)
    sample_masks = torch.zeros(num_samples, num_companies, device=device)
    
    for i in range(num_samples):
        X[i] = eod_data[:, i:i+window_size, :]
        y[i, :, :] = base_price[:, i+window_size:i+window_size+prediction_horizon]
        
        # A sample is valid if all days in the window AND the target day are valid
        window_valid = masks[:, i:i+window_size].min(dim=1)[0]  # [num_companies]
        target_valid = masks[:, i+window_size:i+window_size+prediction_horizon].min(dim=1)[0]
        sample_masks[i] = window_valid * target_valid
    
    return X, y, sample_masks


def temporal_train_val_test_split(X, y, masks):
    num_samples = X.shape[0]
    
    # Calculate split indices based on time
    # train_end = int(num_samples * train_ratio)
    # val_end = int(num_samples * (train_ratio + val_ratio))

    train_end = train_size
    val_end = train_size + validation_size
    # Split data temporally
    splits = {
        'X_train': X[:train_size],
        'y_train': y[:train_size],
        'masks_train': masks[:train_size],
        
        'X_val': X[train_size:train_size+validation_size],
        'y_val': y[train_size:train_size+validation_size],
        'masks_val': masks[train_size:train_size+validation_size],
        
        'X_test': X[train_size+validation_size:],
        'y_test': y[train_size+validation_size:],
        'masks_test': masks[train_size+validation_size:],
    }
    
    # Print split information
    # TODO check this printing
    print(f"\nTemporal Data Split:")
    print(f"  Train: samples [0:{train_end}] = {splits['X_train'].shape[0]} timesteps")
    print(f"  Val:   samples [{train_end}:{val_end}] = {splits['X_val'].shape[0]} timesteps")
    print(f"  Test:  samples [{val_end}:{num_samples}] = {splits['X_test'].shape[0]} timesteps")

    print(f"\n  Valid train samples: {splits['masks_train'].sum().item():.0f} / {splits['masks_train'].numel()}")
    print(f"  Valid val samples:   {splits['masks_val'].sum().item():.0f} / {splits['masks_val'].numel()}")
    print(f"  Valid test samples:  {splits['masks_test'].sum().item():.0f} / {splits['masks_test'].numel()}")
    
    return splits


def adjacency_to_edges(adjacency_matrix, industry_adj=None, wiki_adj=None):
    """
    Convert adjacency matrix to edge_index and edge_weight for PyTorch Geometric
    Supports both 1D (combined) and 2D (separate industry/wiki) edge features
    
    Args:
        adjacency_matrix: [num_companies, num_companies] tensor - combined adjacency
        industry_adj: [num_companies, num_companies] tensor - industry adjacency (optional)
        wiki_adj: [num_companies, num_companies] tensor - wiki adjacency (optional)
        
    Returns:
        edge_index: [2, num_edges] - edge connectivity
        edge_weight: [num_edges, 1] or [num_edges, 2] - edge weights
    """
    adj_np = adjacency_matrix.cpu().numpy()
    rows, cols = np.where(adj_np > 0)
    
    edge_index = torch.tensor(np.stack([rows, cols]), dtype=torch.long)
    
    # Check if we need 2D edge features
    if MODEL_CONFIG['separate_edge_weights'] and industry_adj is not None and wiki_adj is not None:
        # 2D edge features: [industry_weight, wiki_weight]
        industry_np = industry_adj.cpu().numpy()
        wiki_np = wiki_adj.cpu().numpy()
        
        industry_weights = industry_np[rows, cols]
        wiki_weights = wiki_np[rows, cols]
        
        edge_weight = torch.tensor(
            np.stack([industry_weights, wiki_weights], axis=1),  # [num_edges, 2]
            dtype=torch.float32
        )
        print(f"  Using 2D edge features: [industry, wiki]")
    else:
        # 1D edge features: combined weight
        edge_weights = adj_np[rows, cols]
        edge_weight = torch.tensor(edge_weights, dtype=torch.float32).view(-1, 1)  # [num_edges, 1]
        print(f"  Using 1D edge features: combined")
    
    return edge_index, edge_weight


def create_batched_edges(edge_index, edge_weight, num_companies, max_batch_size):
    """
    Creates one big graph with disconnected subgraphs for each timestep
    
    Args:
        edge_index: [2, num_edges] - base edge connectivity
        edge_weight: [num_edges, edge_dim] - base edge weights
        num_companies: Number of nodes per timestep
        max_batch_size: Maximum number of timesteps to support
    
    Returns:
        edge_index_batched: [2, max_batch_size * num_edges]
        edge_weight_batched: [max_batch_size * num_edges, edge_dim]
    """
    num_edges = edge_index.shape[1]
    edge_dim = edge_weight.shape[1]
    
    # Pre-allocate tensors
    edge_index_batched = torch.zeros(2, max_batch_size * num_edges, dtype=torch.long, device=edge_index.device)
    edge_weight_batched = torch.zeros(max_batch_size * num_edges, edge_dim, dtype=torch.float32, device=edge_weight.device)
    
    # Fill in edges for each timestep
    for b in range(max_batch_size):
        start_idx = b * num_edges
        end_idx = (b + 1) * num_edges
        
        # Offset edge indices by b * num_companies
        edge_index_batched[:, start_idx:end_idx] = edge_index + (b * num_companies)
        edge_weight_batched[start_idx:end_idx] = edge_weight
    
    return edge_index_batched, edge_weight_batched


def calculate_loss(predictions, targets, masks, criterion):
    """
    Calculate masked loss (only compute loss on valid samples)
    
    Args:
        predictions: Model predictions [batch, companies, output_dim]
        targets: Ground truth [batch, companies, output_dim]
        masks: Valid sample mask [batch, companies]
        criterion: Loss function (should have reduction='none')
    
    Returns:
        loss: Scalar loss value
    """
    loss_per_sample = criterion(predictions, targets)
    masked_loss = loss_per_sample * masks.unsqueeze(-1)
    num_valid = masks.sum() + 1e-8
    loss = masked_loss.sum() / num_valid
    return loss


print("Utility functions loaded successfully!")

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
# ============================================================================
# CONVERT DATA TO TENSORS
# ============================================================================

print(f"Converting data to tensors for {num_companies} companies...")

# Convert to tensors - use the already subsampled data
eod_data_tensor = torch.tensor(eod_data, dtype=torch.float32)
masks_tensor = torch.tensor(eod_masks, dtype=torch.float32)
price_prediction = torch.tensor(eod_base_price, dtype=torch.float32)

# Relation data tensors
industry_encodings_tensor = torch.tensor(industry_encodings, dtype=torch.float32)
industry_mask_tensor = torch.tensor(industry_mask, dtype=torch.float32)
wiki_encodings_tensor = torch.tensor(wiki_encodings, dtype=torch.float32)
wiki_mask_tensor = torch.tensor(wiki_mask, dtype=torch.float32)

print(f"\nTensor shapes:")
print(f"  EOD data: {eod_data_tensor.shape}")
print(f"  Masks: {masks_tensor.shape}")
print(f"  Price prediction: {price_prediction.shape}")
print(f"  Industry encodings: {industry_encodings_tensor.shape}")
print(f"  Wiki encodings: {wiki_encodings_tensor.shape}")

In [ ]:
# Build graph structures
adjacency_matrix = build_adjacency_matrix(
    industry_encodings_tensor, industry_mask_tensor,
    wiki_encodings_tensor, wiki_mask_tensor,
    device=device
)

# Build separate adjacency matrices if using 2D edge weights
if BASE_MODEL_CONFIG['separate_edge_weights']:
    industry_adj, wiki_adj = build_separate_adjacency_matrices(
        industry_encodings_tensor, industry_mask_tensor,
        wiki_encodings_tensor, wiki_mask_tensor,
        device=device
    )
else:
    industry_adj, wiki_adj = None, None

# Prepare temporal data with sliding windows
X_all, y_all, masks_all = prepare_data(
    eod_data_tensor, masks_tensor, price_prediction,
    window_size=window_size,
    prediction_horizon=prediction_horizon,
    device=device
)

print(f"\nFull dataset prepared:")
print(f"  X_all: {X_all.shape}")
print(f"  y_all: {y_all.shape}")
print(f"  masks_all: {masks_all.shape}")

# Split data temporally into train/val/test
data_splits = temporal_train_val_test_split(
    X_all, y_all, masks_all)

# Extract splits for easier access
X_train = data_splits['X_train']
y_train = data_splits['y_train']
masks_train = data_splits['masks_train']

X_val = data_splits['X_val']
y_val = data_splits['y_val']
masks_val = data_splits['masks_val']

X_test = data_splits['X_test']
y_test = data_splits['y_test']
masks_test = data_splits['masks_test']

# Convert adjacency to edge format for GAT models
print(f"\nConverting adjacency to edge format")
edge_index, edge_weight = adjacency_to_edges(adjacency_matrix, industry_adj, wiki_adj)
edge_index = edge_index.to(device)
edge_weight = edge_weight.to(device)

# Pre-compute batched edges
if BASE_MODEL_CONFIG['use_graph_concatenation']:
    # Calculate max timesteps needed across all configurations
    rnn_max_timesteps = the_max_batch_size * window_size  # is training samples * time steps, which results in the number of disconnected graphs used
    non_rnn_max_timesteps = X_all.shape[0]  # All training samples since without rnn, we just flatten the timesteps * features. so we just have node features. And we have a graph per training sample.
    
    # Use the max to support multiple configs
    max_timesteps = max(rnn_max_timesteps, non_rnn_max_timesteps)
    
    print(f"  RNN mode would need: {rnn_max_timesteps} timesteps")
    print(f"  Non-RNN mode would need: {non_rnn_max_timesteps} timesteps")
    print(f"  Allocating for MAX: {max_timesteps} timesteps (supports both modes)")

    edge_index_batched, edge_weight_batched = create_batched_edges(
        edge_index, edge_weight, num_companies, max_timesteps
    )
    print(f"  Edge index batched: {edge_index_batched.shape}")
    print(f"  Edge weight batched: {edge_weight_batched.shape}")
    print(f"  Memory allocated: ~{(edge_index_batched.numel() * 8 + edge_weight_batched.numel() * 4) / 1024 / 1024:.1f} MB")
else:
    edge_index_batched, edge_weight_batched = None, None

print(f"\n{'='*60}")
print(f"Graph structure:")
print(f"  Adjacency matrix: {adjacency_matrix.shape}")
print(f"  Edge index: {edge_index.shape}")
print(f"  Edge weight: {edge_weight.shape}")
print(f"  Edge dimension: {edge_weight.shape[1]}D ({'industry+wiki separate' if edge_weight.shape[1] == 2 else 'combined'}) ")
print(f"  Total edges: {edge_index.shape[1]}")
print(f"{'='*60}")

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()


# GATModel

In [ ]:
class GATModel(nn.Module):
    """
   Options:
    1. Without RNN: GAT(flattened_time) → prediction
    2. With RNN: GAT(t=1) → ... → GAT(t=20) → RNN → prediction
    3. With RNN first, so RNN -> GAT(one embedding per window) -> prediction

    """
    def __init__(self, layers_dim, num_companies, adjacency_matrix, device, edge_index, edge_weight,
                 edge_index_batched=None, edge_weight_batched=None, window_size=20):
        super(GATModel, self).__init__()
        self.device = device
        self.num_companies = num_companies
        self.layers_dim = layers_dim
        self.window_size = window_size
        
        # Graph structure
        self.adjacency_matrix = adjacency_matrix
        self.mask = (self.adjacency_matrix == 0)
        self.edge_index = edge_index
        self.edge_weight = edge_weight
        
        # Batched edges (for fast processing)
        self.use_batched = MODEL_CONFIG['use_graph_concatenation'] and edge_index_batched is not None
        if self.use_batched:
            self.register_buffer('edge_index_batched', edge_index_batched)
            self.register_buffer('edge_weight_batched', edge_weight_batched)
        
        # Determine edge dimension (1D or 2D)
        edge_dim = edge_weight.shape[1]
        
        # Temporal modeling configuration
        self.use_rnn = MODEL_CONFIG['use_rnn']
        
        if self.use_rnn:
            # ===== WITH RNN: GAT outputs embeddings, RNN processes them temporally =====
            gat_output_dim = MODEL_CONFIG['hidden_dim']
            
            self.conv_0 = GATConv(
                in_channels=layers_dim[0][0],  # num_features (5)
                out_channels=gat_output_dim,
                heads=MODEL_CONFIG['num_attention_heads'],
                concat=False,
                dropout=MODEL_CONFIG['dropout'],
                add_self_loops=True,
                edge_dim=edge_dim,
                fill_value=0,
                bias=True
            )
            
            # RNN processes temporal embeddings
            rnn_input_dim = gat_output_dim
            rnn_hidden_dim = MODEL_CONFIG['rnn_hidden_dim']
            rnn_num_layers = MODEL_CONFIG['rnn_num_layers']

            if MODEL_CONFIG['rnn_type'] == 'LSTM':
                self.rnn = nn.LSTM(
                    input_size=rnn_input_dim,
                    hidden_size=rnn_hidden_dim,
                    num_layers=rnn_num_layers,
                    batch_first=True,
                    bidirectional=False,
                    dropout=MODEL_CONFIG['dropout'] if rnn_num_layers > 1 else 0
                )
            else:  # GRU
                self.rnn = nn.GRU(
                    input_size=rnn_input_dim,
                    hidden_size=rnn_hidden_dim,
                    num_layers=rnn_num_layers,
                    batch_first=True,
                    bidirectional=False,
                    dropout=MODEL_CONFIG['dropout'] if rnn_num_layers > 1 else 0
                )
            
            # Final projection from RNN output to prediction
            rnn_output_dim = rnn_hidden_dim * 1
            self.output_projection = nn.Linear(rnn_output_dim, layers_dim[0][1]) # output layer linear
            
        else:
            # ===== WITHOUT RNN/TCN: GAT directly outputs prediction (original) =====
            self.conv_0 = GATConv(
                in_channels=layers_dim[0][0],  # num_features * window_size
                out_channels=layers_dim[0][1], # prediction_horizon
                heads=MODEL_CONFIG['num_attention_heads'],
                concat=False,
                dropout=MODEL_CONFIG['dropout'],
                add_self_loops=True,
                edge_dim=edge_dim,
                fill_value=0,
                bias=True
            )
        
        # Optional ReLU activation
        self.use_relu = MODEL_CONFIG['use_relu_output']
        
        # Store config for forward pass
        self.rnn_type = MODEL_CONFIG.get('rnn_type', 'LSTM')

        self.use_rnn_first = MODEL_CONFIG['use_rnn_first']
    
    def forward(self, x):
        """
        Args:
            x: Historical data [batch, num_companies, window_size, num_features]
        Returns:
            predictions: [batch, num_companies, output_dim]
        """
        batch_size = x.shape[0] # normally 4 now, number of training samples
        
        if self.use_rnn:
            if self.use_rnn_first:
                x_rnn_in = x.view(-1, self.window_size, x.shape[-1])  # [B*C, T, F]
                # RNN forward pass
                if self.rnn_type == 'LSTM':
                    rnn_out, (h_n, c_n) = self.rnn(x_rnn_in)
                else:  # GRU
                    rnn_out, h_n = self.rnn(x_rnn_in)

                # Use last hidden state
                h_n = h_n[-1]
                lstm_embeddings = h_n.view(batch_size, self.num_companies, -1)

                if self.use_batched:
                    x_flat = lstm_embeddings.view(-1, lstm_embeddings.shape[-1]) # does training samples * companies, so you get for every node the features from the lstm
                    num_edges_per_graph = self.edge_index.shape[1] # is just the normal number of edges in the graph , edge index has shape [2, num_edges]
                    total_edges = batch_size * num_edges_per_graph
                    edge_idx = self.edge_index_batched[:, :total_edges]
                    edge_wgt = self.edge_weight_batched[:total_edges]
                    out = self.conv_0(x_flat, edge_idx, edge_wgt)
                    out = out.view(batch_size, self.num_companies, -1)

                else:
                    outputs = []
                    for b in range(batch_size):
                        out_b = self.conv_0(lstm_embeddings[b], self.edge_index, self.edge_weight)
                        outputs.append(out_b)
                    out = torch.stack(outputs)
            else:
                # ===== WITH RNN: Compute GAT embeddings for each timestep =====
                # (RNN implementation)
                if self.use_batched:
                    # FAST: Process all batch*window_size timesteps together
                    x_reshaped = x.permute(0, 2, 1, 3).contiguous() # reorder to (training samples, time, companies, features) # todo maybe remove contiguous is maybe not needed
                    x_flat = x_reshaped.view(-1, self.num_companies, x.shape[-1])  # create training_samples*timesteps, companies, features, so you have training_samples*timesteps number of graphs
                    x_gat_input = x_flat.view(-1, x.shape[-1]) # now training_samples*timesteps*companies, features -> (training_samples*timesteps) number of subgraphs, now times the companies

                    # since we now have training_samples*timesteps number of subgraphs, we need that many subgraph edge indices and weights
                    # to select the right ones we do the following
                    num_edges_per_timestep = self.edge_index.shape[1] # is just the normal number of edges in the graph , edge index has shape [2, num_edges]
                    total_timesteps = batch_size * self.window_size # this is just training samples * timesteps, so we have total_timesteps or total number of subgraphs
                    total_edges = total_timesteps * num_edges_per_timestep # just checking how many edges in total if we have total timesteps number of subgraphs

                    edge_idx = self.edge_index_batched[:, :total_edges] # shape of edge_index_batched is [2, total_edges], now you keep the edges you need for all the subgraphs together, todo this is kinda not really needed since all graphs are the same but okay so can improve this still
                    edge_wgt = self.edge_weight_batched[:total_edges] # here the same but just take the weights

                    embeddings = self.conv_0(x_gat_input, edge_idx, edge_wgt) # run gatconv with (training samples * timesteps) disconnected graphs
                    embeddings = embeddings.view(batch_size, self.window_size, self.num_companies, -1) # now turn back to training samples , timesteps, companies, features(features are here the output of gat)
                    embeddings = embeddings.permute(0, 2, 1, 3).contiguous() # back to training samples, companies, timesteps, features
                else:
                    # SLOW: Process each timestep sequentially
                    timestep_embeddings = []
                    for t in range(self.window_size): # need to do per timestep because we need per timestep for the RNN
                        x_t = x[:, :, t, :] # pick only current timestep for all training samples and companies, and pick all features
                        batch_embeddings = []
                        for b in range(batch_size): # for each training sample
                            emb = self.conv_0(x_t[b], self.edge_index, self.edge_weight)
                            batch_embeddings.append(emb)
                        timestep_embeddings.append(torch.stack(batch_embeddings))
                    embeddings = torch.stack(timestep_embeddings, dim=2)

                # embeddings: [batch, companies, window_size, hidden_dim]
                embeddings_rnn = embeddings.view(-1, self.window_size, embeddings.shape[-1]) # so will run per company

                # RNN forward pass
                if self.rnn_type == 'LSTM':
                    rnn_out, (h_n, c_n) = self.rnn(embeddings_rnn)
                else:  # GRU
                    rnn_out, h_n = self.rnn(embeddings_rnn)

                # Use last hidden state
                h_n = h_n[-1]

                # Project to output
                out = self.output_projection(h_n)
                out = out.view(batch_size, self.num_companies, -1)
            
        else:
            # ===== WITHOUT RNN: Original architecture (flatten time) =====
            x_reshaped = x.view(batch_size, self.num_companies, -1)
            
            if self.use_batched:
                # FAST BATCHED PROCESSING
                x_flat = x_reshaped.view(-1, x_reshaped.shape[-1]) # gives (training samples * companies) which all have features
                
                num_edges_per_timestep = self.edge_index.shape[1] # is just the normal number of edges in the graph , edge index has shape [2, num_edges]
                total_edges = batch_size * num_edges_per_timestep # total edges when accoutning for the batch size
                
                edge_idx = self.edge_index_batched[:, :total_edges]
                edge_wgt = self.edge_weight_batched[:total_edges]
                
                out = self.conv_0(x_flat, edge_idx, edge_wgt)
                out = out.view(batch_size, self.num_companies, -1) # convert back to batch, companies, features
            else:
                # SLOW SEQUENTIAL PROCESSING
                outputs = []
                for b in range(batch_size): # every training sample separate
                    out_b = self.conv_0(x_reshaped[b], self.edge_index, self.edge_weight) # output of gat layer
                    outputs.append(out_b)
                out = torch.stack(outputs)
        
        # Optional ReLU activation (off by default for regression)
        if self.use_relu:
            out = F.relu(out)
        
        return out


# Grid search

In [ ]:
import os
os.makedirs('models', exist_ok=True)

# Storage for results
grid_search_results = []
all_epoch_losses = {}  # Store per-epoch losses for plotting

print(f"\n{'='*80}")
print(f"start grid search - {len(CONFIGS_TO_TEST)} configs")
print(f"{'='*80}\n")

for config_idx, config_updates in enumerate(CONFIGS_TO_TEST, 1):
    print(f"\n{'#'*80}")
    print(f"# configs {config_idx}/{len(CONFIGS_TO_TEST)}: {config_updates['name']}")
    print(f"{'#'*80}\n")

    # Update MODEL_CONFIG with this configuration
    MODEL_CONFIG = BASE_MODEL_CONFIG.copy()
    MODEL_CONFIG.update(config_updates)

    # Print configuration
    print(f"Configuration details:")
    for key, value in config_updates.items():
        if key != 'name':
            print(f"  {key}: {value}")

    # Clear GPU memory
    torch.cuda.empty_cache()

    # Verify features
    print(f"\nFeature verification:")
    print(f"  Input shape: {X_train.shape}")
    print(f"  Features per timestep: {X_train.shape[3]} (should be {num_features})")
    assert X_train.shape[3] == num_features, f"Expected {num_features} features but got {X_train.shape[3]}"
    print(f"  All {num_features} features are being used!")

    # Initialize model
    print(f"\nInitializing model...")
    model = GATModel(
        layers_dim=[(num_features * window_size, prediction_horizon)] if (not MODEL_CONFIG['use_rnn'])
                   else [(num_features, MODEL_CONFIG['rnn_hidden_dim'])],
        num_companies=num_companies,
        adjacency_matrix=adjacency_matrix,
        device=device,
        edge_index=edge_index,
        edge_weight=edge_weight,
        edge_index_batched=edge_index_batched if MODEL_CONFIG['use_graph_concatenation'] else None,
        edge_weight_batched=edge_weight_batched if MODEL_CONFIG['use_graph_concatenation'] else None,
        window_size=window_size,
    ).to(device)

    # Training configuration
    criterion = nn.MSELoss(reduction='none')
    optimizer = optim.Adam(
        model.parameters(),
        lr=MODEL_CONFIG['learning_rate'],
        weight_decay=MODEL_CONFIG['weight_decay']
    )

    print(f"Model details:")
    if MODEL_CONFIG['use_rnn']:
        if MODEL_CONFIG['use_rnn_first']:
            print(f"  Architecture: RNN → GAT → Prediction")
        else:
            print(f"  Architecture: GAT → RNN → Prediction")
        print(f"  RNN type: {MODEL_CONFIG['rnn_type']}")
        print(f"  RNN hidden dim: {MODEL_CONFIG['rnn_hidden_dim']}")
    else:
        print(f"  Architecture: GAT → Prediction (flattened time)")
    print(f"  Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"  Learning rate: {MODEL_CONFIG['learning_rate']}")

    # Training tracking
    best_val_loss = float('inf')
    patience_counter = 0 #
    train_losses = []
    val_losses = []
    test_losses = []
    batch_size = MODEL_CONFIG['batch_size']

    print(f"\n{'-'*60}")
    print(f"Starting training for {epochs} epochs...")
    print(f"{'-'*60}\n")

    # Training loop
    for epoch in range(epochs):
        model.train()
        num_training_samples = X_train.shape[0]
        total_train_loss = 0
        num_batches = 0

        for i in range(0, num_training_samples, batch_size): # do it in batches
            optimizer.zero_grad()
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            batch_masks = masks_train[i:i+batch_size]
            predictions = model(batch_X)

            train_loss = calculate_loss(predictions, batch_y, batch_masks, criterion)
            train_loss.backward()
            optimizer.step()

            total_train_loss += train_loss.item()
            num_batches += 1

        avg_train_loss = total_train_loss / num_batches
        train_losses.append(avg_train_loss)

        # ===== VALIDATION =====
        model.eval()
        total_val_loss = 0
        num_val_batches = 0

        with torch.no_grad():
            for i in range(0, X_val.shape[0], batch_size):
                batch_X = X_val[i:i+batch_size]
                batch_y = y_val[i:i+batch_size]
                batch_masks = masks_val[i:i+batch_size]
                val_predictions = model(batch_X)
                val_loss = calculate_loss(val_predictions, batch_y, batch_masks, criterion)
                total_val_loss += val_loss.item()
                num_val_batches += 1

        avg_val_loss = total_val_loss / num_val_batches
        val_losses.append(avg_val_loss)

        # test
        total_test_loss = 0
        num_test_batches = 0

        with torch.no_grad():
            for i in range(0, X_test.shape[0], batch_size):
                batch_X = X_test[i:i+batch_size]
                batch_y = y_test[i:i+batch_size]
                batch_masks = masks_test[i:i+batch_size]
                test_predictions = model(batch_X)
                test_loss = calculate_loss(test_predictions, batch_y, batch_masks, criterion)
                total_test_loss += test_loss.item()
                num_test_batches += 1

        avg_test_loss = total_test_loss / num_test_batches
        test_losses.append(avg_test_loss)

        # Print progress
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1:3d}/{epochs}] | Train: {avg_train_loss:.6f} | Val: {avg_val_loss:.6f} | Test: {avg_test_loss:.6f}')

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1

        if patience_counter >= early_stopping_patience:
            print(f"\nEarly stopping triggered after {epoch+1} epochs")
            break

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Load best model
    model.load_state_dict(best_model_state)

    print(f"\n{'-'*60}")
    print(f"FINAL EVALUATION")
    print(f"{'-'*60}")

    model.eval()
    with torch.no_grad():
        # Train set
        all_train_preds = []
        for i in range(0, X_train.shape[0], batch_size):
            batch_X = X_train[i:i+batch_size]
            all_train_preds.append(model(batch_X))
        train_predictions = torch.cat(all_train_preds, dim=0)
        final_train_loss = calculate_loss(train_predictions, y_train, masks_train, criterion)

        # Validation set
        all_val_preds = []
        for i in range(0, X_val.shape[0], batch_size):
            batch_X = X_val[i:i+batch_size]
            all_val_preds.append(model(batch_X))
        val_predictions = torch.cat(all_val_preds, dim=0)
        final_val_loss = calculate_loss(val_predictions, y_val, masks_val, criterion)

        # Test set
        all_test_preds = []
        for i in range(0, X_test.shape[0], batch_size):
            batch_X = X_test[i:i+batch_size]
            all_test_preds.append(model(batch_X))
        test_predictions = torch.cat(all_test_preds, dim=0)
        final_test_loss = calculate_loss(test_predictions, y_test, masks_test, criterion)

    # Calculate RMSE
    train_rmse = torch.sqrt(final_train_loss).item()
    val_rmse = torch.sqrt(final_val_loss).item()
    test_rmse = torch.sqrt(final_test_loss).item()

    print(f"\nFinal RMSE:")
    print(f"  Train: {train_rmse:.6f}")
    print(f"  Val:   {val_rmse:.6f}")
    print(f"  Test:  {test_rmse:.6f}")

    # Store per-epoch losses for this model
    model_name = config_updates['name']
    all_epoch_losses[model_name] = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'test_losses': test_losses,
    }

    # Save results
    result = {
        'name': config_updates['name'],
        'train_rmse': train_rmse,
        'val_rmse': val_rmse,
        'test_rmse': test_rmse,
        'train_mse': final_train_loss.item(),
        'val_mse': final_val_loss.item(),
        'test_mse': final_test_loss.item(),
        'epochs_trained': len(train_losses),
        'config': config_updates.copy(),
        'train_pred': train_predictions.cpu().numpy(),
        'val_pred': val_predictions.cpu().numpy(),
        'test_pred': test_predictions.cpu().numpy(),
    }
    grid_search_results.append(result)

    # Save model checkpoint
    model_path = f"models/{config_updates['name']}.pt"
    torch.save(model.state_dict(), model_path)
    print(f"\nModel saved to: {model_path}")

    # Clean up
    del model, optimizer, criterion
    torch.cuda.empty_cache()

    print(f"\n{'#'*80}\n")

# ============================================================================
# grid ssearch summary

print(f"\n{'='*80}")
print(f"GRID SEARCH COMPLETE - RESULTS SUMMARY")
print(f"{'='*80}\n")

# Convert to DataFrame for easy viewing
import pandas as pd
results_df = pd.DataFrame(grid_search_results)
results_df = results_df.sort_values('val_rmse')

print("Results sorted by validation RMSE (best first):\n")
print(results_df[['name', 'train_rmse', 'val_rmse', 'test_rmse', 'epochs_trained']].to_string(index=False))

print(f"\n{'-'*80}")
print(f"BEST MODEL: {results_df.iloc[0]['name']}")
print(f"  Train RMSE: {results_df.iloc[0]['train_rmse']:.6f}")
print(f"  Val RMSE:   {results_df.iloc[0]['val_rmse']:.6f}")
print(f"  Test RMSE:  {results_df.iloc[0]['test_rmse']:.6f}")
print(f"{'-'*80}")

# Save results to CSV
results_df.to_csv('grid_search_results.csv', index=False)
print(f"\nResults saved to: grid_search_results.csv")
print(f"{'='*80}\n")

# plot + eval

## Baseline model

In [ ]:
# For baseline: prediction = last price in the window
# X_train shape: [num_samples, num_companies, window_size, num_features]
# y_train shape: [num_samples, num_companies, prediction_horizon]

def evaluate_baseline(X, y, masks):
    """Baseline: predict tomorrow's price = today's closing price"""
    # Last price in window is X[:, :, -1, -1] (last timestep, last feature = close price)
    baseline_predictions = X[:, :, -1, -1].unsqueeze(-1)  # [batch, companies, 1]

    # Calculate loss
    criterion = nn.MSELoss(reduction='none')
    loss = calculate_loss(baseline_predictions, y, masks, criterion)
    rmse = torch.sqrt(loss).item()

    return loss.item(), rmse, baseline_predictions

# Evaluate baseline on all splits
baseline_train_mse, baseline_train_rmse, baseline_train_pred = evaluate_baseline(X_train, y_train, masks_train)
baseline_val_mse, baseline_val_rmse, baseline_val_pred = evaluate_baseline(X_val, y_val, masks_val)
baseline_test_mse, baseline_test_rmse, baseline_test_pred = evaluate_baseline(X_test, y_test, masks_test)

print(f"\nBaseline Model Results:")
print(f"  Train RMSE: {baseline_train_rmse:.6f}")
print(f"  Val RMSE:   {baseline_val_rmse:.6f}")
print(f"  Test RMSE:  {baseline_test_rmse:.6f}")

# Store baseline results for plotting
baseline_result = {
    'name': 'Baseline (tomorrow=today)',
    'train_rmse': baseline_train_rmse,
    'val_rmse': baseline_val_rmse,
    'test_rmse': baseline_test_rmse,
    'train_mse': baseline_train_mse,
    'val_mse': baseline_val_mse,
    'test_mse': baseline_test_mse,
    'epochs_trained': 0,
    'train_pred': baseline_train_pred.cpu().numpy(),
    'val_pred': baseline_val_pred.cpu().numpy(),
    'test_pred': baseline_test_pred.cpu().numpy(),
}

## Create graph with on the x axis the number of epochs and the loss on the y axi

In [ ]:
if 'all_epoch_losses' in locals() and len(all_epoch_losses) > 0:
    import matplotlib.pyplot as plt
    import numpy as np

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Plot 1: Training Loss
    ax1 = axes[0]
    for model_name, losses in all_epoch_losses.items():
        epochs_range = range(1, len(losses['train_losses']) + 1)
        ax1.plot(epochs_range, losses['train_losses'], label=model_name, alpha=0.7, linewidth=2)

    # Add baseline (horizontal line since it doesn't train)
    ax1.axhline(y=baseline_train_mse, color='black', linestyle='--',
                linewidth=2, label='Baseline (tomorrow=today)', alpha=0.8)

    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('MSE Loss', fontsize=12)
    ax1.set_title(f'Training Loss vs Epoch - {market}', fontsize=14, fontweight='bold')
    ax1.legend(loc='best', fontsize=9)
    ax1.grid(True, alpha=0.3)

    # Plot 2: Validation Loss
    ax2 = axes[1]
    for model_name, losses in all_epoch_losses.items():
        epochs_range = range(1, len(losses['val_losses']) + 1)
        ax2.plot(epochs_range, losses['val_losses'], label=model_name, alpha=0.7, linewidth=2)

    # Add baseline
    ax2.axhline(y=baseline_val_mse, color='black', linestyle='--',
                linewidth=2, label='Baseline (tomorrow=today)', alpha=0.8)

    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('MSE Loss', fontsize=12)
    ax2.set_title(f'Validation Loss vs Epoch - {market}', fontsize=14, fontweight='bold')
    ax2.legend(loc='best', fontsize=9)
    ax2.grid(True, alpha=0.3)

    # Plot 3: Test Loss
    ax3 = axes[2]
    for model_name, losses in all_epoch_losses.items():
        epochs_range = range(1, len(losses['test_losses']) + 1)
        ax3.plot(epochs_range, losses['test_losses'], label=model_name, alpha=0.7, linewidth=2)

    # Add baseline
    ax3.axhline(y=baseline_test_mse, color='black', linestyle='--',
                linewidth=2, label='Baseline (tomorrow=today)', alpha=0.8)

    ax3.set_xlabel('Epoch', fontsize=12)
    ax3.set_ylabel('MSE Loss', fontsize=12)
    ax3.set_title(f'Test Loss vs Epoch - {market}', fontsize=14, fontweight='bold')
    ax3.legend(loc='best', fontsize=9)
    ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{market}_epoch_vs_loss_all_models.png', dpi=150, bbox_inches='tight')
    plt.show()

    print(f"\nVisualization saved to: {market}_epoch_vs_loss_all_models.png")
else:
    print("No epoch losses available. Please run the grid search first.")

# Plot predition function

In [ ]:
def plot_predictions(pred_price, actual_price,
                     start_idx, num_companies_to_plot=5):
    """
    Plot predicted vs actual stock prices for random companies

    Args:
        pred_price: Predicted prices [num_samples, num_companies, prediction_horizon]
        actual_price: Actual prices [num_samples, num_companies, prediction_horizon]
        start_idx: Starting day index in the original timeline
        num_companies_to_plot: Number of companies to visualize
    """
    import matplotlib.pyplot as plt
    import numpy as np

    num_samples, num_companies, prediction_horizon = pred_price.shape

    # Select 5 random companies
    np.random.seed(40)
    selected_companies = np.random.choice(num_companies, num_companies_to_plot, replace=False)

    # Create subplots
    fig, axes = plt.subplots(num_companies_to_plot, 1, figsize=(12, 3*num_companies_to_plot))
    if num_companies_to_plot == 1:
        axes = [axes]

    for idx, company_idx in enumerate(selected_companies):
        ax = axes[idx]

        actual_prices = actual_price[:, company_idx, 0]
        pred_prices = pred_price[:, company_idx, 0]
        time_steps = np.arange(start_idx, start_idx + len(actual_prices))

        # Plot
        ax.plot(time_steps, actual_prices, label='Actual Price',
                color='blue', linewidth=2, alpha=0.7)
        ax.plot(time_steps, pred_prices, label='Predicted Price',
                color='red', linewidth=2, linestyle='--', alpha=0.7)

        ax.set_xlabel('Day Index')
        ax.set_ylabel('Price')
        ax.set_title(f'Company {company_idx}: Predicted vs Actual Stock Price')
        ax.legend()
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('stock_predictions.png', dpi=150, bbox_inches='tight')
    print("Plot saved as 'stock_predictions.png'")
    plt.show()

print("Plot predictions function loaded!")

In [ ]:
# ============================================================================
# Plot predictions for best model
# ============================================================================

if 'grid_search_results' in locals() and len(grid_search_results) > 0:
    # Get best model (lowest validation RMSE)
    best_model = min(grid_search_results, key=lambda x: x['val_rmse'])
    
    print(f"Plotting predictions for best model: {best_model['name']}")
    print(f"Test RMSE: {best_model['test_rmse']:.6f}\n")
    
    # Get predictions and actual prices for test set
    pred_test = best_model['test_pred']  # [num_samples, num_companies, prediction_horizon]
    actual_test = y_test.cpu().numpy()
    
    # Calculate starting index for test set (after train and val)
    train_end = train_size
    val_end = train_size + validation_size
    test_start_idx = val_end + window_size  # todo check this ???
    
    # Plot predictions
    plot_predictions(pred_test, actual_test, 
                     start_idx=test_start_idx, 
                     num_companies_to_plot=5)

# Saving all

In [ ]:
#Saving all

import numpy as np
import pickle

# Prepare actual prices (ground truth)
actual_prices = {
    'train': y_train.cpu().numpy(),  # [num_samples, num_companies, prediction_horizon]
    'val': y_val.cpu().numpy(),
    'test': y_test.cpu().numpy(),
}

# Prepare all model predictions (including baseline)
all_predictions = {}

# Add baseline
all_predictions['Baseline (tomorrow=today)'] = {
    'train': baseline_result['train_pred'],
    'val': baseline_result['val_pred'],
    'test': baseline_result['test_pred'],
}

# Add all grid search models
for result in grid_search_results:
    all_predictions[result['name']] = {
        'train': result['train_pred'],
        'val': result['val_pred'],
        'test': result['test_pred'],
    }

# Save to file for portfolio evaluation
save_data = {
    'actual_prices': actual_prices,
    'predictions': all_predictions,
    'market': market,
    'num_companies': num_companies,
    'window_size': window_size,
    'train_size': train_size,
    'validation_size': validation_size,
    'test_size': test_size,
}

# Save as pickle for easy loading
with open(f'{market}_predictions_and_actuals.pkl', 'wb') as f:
    pickle.dump(save_data, f)

print(f"\n{'='*60}")
print(f"SAVED PREDICTIONS FOR PORTFOLIO EVALUATION")
print(f"{'='*60}")
print(f"File: {market}_predictions_and_actuals.pkl")
print(f"\nContents:")
print(f"  - Actual prices (train/val/test)")
print(f"  - Predictions for {len(all_predictions)} models")
print(f"  - Metadata (market, num_companies, splits, etc.)")
print(f"\nTo load in portfolio code:")
print(f"  import pickle")
print(f"  with open('{market}_predictions_and_actuals.pkl', 'rb') as f:")
print(f"      data = pickle.load(f)")
print(f"  actual_test = data['actual_prices']['test']")
print(f"  pred_test = data['predictions']['GAT_LSTM']['test']")
print(f"{'='*60}\n")